In [1]:
import pandas as pd

# Landsat Feature Extraction


In [3]:
df_landsat_features_train = pd.read_csv("landsat_features_training_all_bands.csv")
display(df_landsat_features_train.head(5))

,Unnamed: 0,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus,qa,red,blue,...,cdist,green,nir08,lwir.1,swir16,swir22,cloud_qa,qa_pixel,qa_radsat,atmos_opacity
0,0,-28.760833,17.730278,02-01-2011,128.912,555.0,10.0,172.0,12802.0,9557.0,...,2447.5,11426.0,11190.0,44344.0,7687.5,7645.0,32.0,5504.0,NaN,8.0
1,1,-26.861111,28.884722,03-01-2011,74.720,162.9,163.0,213.0,9241.5,8691.0,...,476.5,9550.0,17658.5,45396.5,13746.5,10574.0,NaN,5440.0,NaN,46.0
2,2,-26.450000,28.085833,03-01-2011,89.254,573.0,80.0,298.0,12540.0,9502.5,...,178.5,10720.0,15210.0,41467.5,17974.0,14201.0,2.0,5440.0,NaN,9.0
3,3,-27.671111,27.236944,03-01-2011,82.000,203.6,101.0,284.0,11237.5,9693.5,...,381.5,10943.0,14887.0,46121.0,13522.0,11403.0,NaN,5440.0,NaN,48.0
4,4,-27.356667,27.286389,03-01-2011,56.100,145.1,151.0,260.0,9290.0,8718.0,...,930.0,9502.5,16828.5,44381.0,12665.5,9643.0,NaN,5440.0,NaN,106.5


In [5]:
df_landsat_features_val = pd.read_csv("landsat_features_val_all_bands.csv")
display(df_landsat_features_val.head(5))

,qa,red,blue,drad,emis,emsd,lwir,trad,urad,atran,cdist,green,nir08,lwir.1,swir16,swir22,cloud_qa,qa_pixel,qa_radsat,atmos_opacity
0,609.0,13210.0,11282.0,441.0,9704.0,39.0,NaN,7226.5,821.0,8729.0,NaN,12868.0,15229.0,NaN,14797.0,12421.0,NaN,22280.0,NaN,NaN
1,-9999.0,NaN,NaN,-4676.5,-9999.0,-9999.0,NaN,-702.0,-4355.0,-827.5,-9999.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,-9999.0
2,203.5,9145.0,8111.5,653.0,9837.0,47.0,NaN,9005.5,1302.0,8352.0,955.5,9304.5,16221.0,NaN,12536.5,9958.0,NaN,21824.0,NaN,NaN
3,-9999.0,NaN,NaN,-9999.0,-9999.0,-9999.0,NaN,-9999.0,-9999.0,-9999.0,-9999.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,-9999.0
4,209.0,11166.0,9504.0,778.0,9824.0,NaN,NaN,9074.5,1577.5,7928.5,738.5,11100.5,9125.0,NaN,9455.0,8711.0,NaN,21952.0,NaN,NaN


In [6]:
df_dict = {
    "train": df_landsat_features_train
    , "val": df_landsat_features_val
}

for df_type in df_dict.keys():
    df_processing = df_dict[df_type]
    ## Vegetation Index (NVDI)
    df_processing["NVDI"] = (df_processing["nir08"] - df_processing["red"]) / (df_processing["nir08"] + df_processing["red"])

    ## Soil Adjusted Vegetation Index (SAVI)
    L = 0.5
    df_processing["SAVI"] = ((df_processing["nir08"] - df_processing["red"]) / (df_processing["nir08"] + df_processing["red"] + L)) * (1 + L)
    ## Bare Soil Index (BSI)
    df_processing["BSI"] = (((df_processing["swir16"] + df_processing["red"]) - (df_processing["nir08"] + df_processing["blue"]))
                            / ((df_processing["swir16"] + df_processing["red"]) + (df_processing["nir08"] + df_processing["blue"])))
    ## Build-Up Index (NDBI)
    df_processing["NDBI"] = (df_processing["swir16"] - df_processing["nir08"]) / (df_processing["swir16"] + df_processing["nir08"])

    ## Tasseled Cap (TC) Wetness Index (TCWI)
    df_processing["TCWI"] = (0.1511 * df_processing["blue"]
                             + 0.1973 * df_processing["green"]
                             + 0.3283 * df_processing["red"]
                             + 0.3407 * df_processing["nir08"]
                             - 0.7117 * df_processing["swir16"]
                             - 0.4559 * df_processing["swir22"])

    df_processing.to_csv(f"landsat_index_features_{df_type}", index=False)

